# Reconocimiento Facial con Machine Learning para Seguridad

En este proyecto se desarrolla un sistema de reconocimiento facial basado en modelos de *machine learning*, cuyo objetivo es identificar personas autorizadas en un entorno de seguridad utilizando cámaras que capturan imágenes en tiempo real. El sistema tomará los frames capturados por las cámaras de vigilancia y los procesará mediante un algoritmo de clasificación que determina si la persona detectada pertenece al conjunto de individuos autorizados o no.

Adicionalmente, se busca extender el análisis para identificar el género de la persona, con el propósito de enriquecer la información capturada por el sistema de seguridad. Esta característica puede ser utilizada como una capa adicional de validación o para realizar estadísticas de acceso por género, según las necesidades del sistema.

## Dataset Utilizado: Labeled Faces in the Wild (LFW)

Para entrenar el modelo se utilizará el dataset **Labeled Faces in the Wild (LFW)**, una base de datos ampliamente utilizada para el estudio del reconocimiento facial en condiciones no controladas (*in the wild*). Este conjunto fue creado y es mantenido por investigadores de la Universidad de Massachusetts, Amherst.

El dataset contiene un total de **13,233 imágenes** de rostros de **5,749 personas**, detectadas y centradas utilizando el detector de rostros de Viola-Jones. Las imágenes fueron recolectadas de la web, y representan variaciones reales de expresión facial, iluminación, pose, edad y fondo.

De las personas incluidas, **1,680** tienen dos o más imágenes distintas en el conjunto, lo cual permite realizar tareas de entrenamiento y validación más efectivas para problemas de clasificación facial.

Las imágenes están disponibles en color y tienen un tamaño de **250x250 píxeles**. Cada una viene etiquetada con el nombre de la persona, aunque el dataset **no incluye información de género**, por lo que será necesario añadir esta información manualmente o mediante alguna fuente auxiliar para tareas de clasificación de género.

Este notebook se basa en el ejemplo oficial de Scikit-learn: [Face recognition example](https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html), al cual se le añaden mejoras y adaptaciones para cumplir con los objetivos del sistema de seguridad.

### Acceso al Dataset

El dataset puede ser accedido de dos formas:

Directamente desde **scikit-learn**, utilizando la función:

  ```python
  from sklearn.datasets import fetch_lfw_people
  lfw_people = fetch_lfw_people()
  ```

Esta opción permite cargar automáticamente las imágenes y etiquetas con un formato listo para usar en modelos de aprendizaje automático.

También puede descargarse desde Kaggle, en el siguiente enlace: https://kaggle.com/datasets/jessicali9530/lfw-dataset

Esta versión contiene las imágenes originales organizadas en carpetas por nombre, útil para procesamiento personalizado o entrenamiento con redes neuronales convolucionales más complejas.


In [ ]:
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt
import numpy as np

# Cargar el dataset (personas con al menos 70 imágenes para mantener equilibrio en las clases)
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4, color=True)

# Obtener datos
n_samples, h, w = lfw_people.images.shape
X = lfw_people.data      # datos de las imágenes (flattened)
y = lfw_people.target    # etiquetas (índices de personas)
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print(f"Número de muestras: {n_samples}")
print(f"Dimensiones de cada imagen: {h} x {w}")
print(f"Número de clases (personas): {n_classes}")
print(f"Personas en el dataset: {target_names}")

# Visualizar las primeras 12 imágenes del dataset
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=0.01, right=0.99, top=0.90, hspace=0.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i], interpolation='nearest')
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

# Generar títulos con los nombres de las personas
titles = [target_names[y[i]] for i in range(12)]
plot_gallery(lfw_people.images, titles, h, w)
plt.show()

In [ ]:
from sklearn.datasets import load_files
from sklearn.utils import Bunch
from PIL import Image
import numpy as np
import os
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/lfw-dataset")

print("Path to dataset files:", path)


def load_custom_lfw(path, image_size=(62, 47)):
    images = []
    flat_data = []
    target = []
    target_names = os.listdir(path)
    label_map = {name: idx for idx, name in enumerate(target_names)}

    for person in target_names:
        person_dir = os.path.join(path, person)
        if not os.path.isdir(person_dir):
            continue
        for file in os.listdir(person_dir):
            if file.endswith(".jpg"):
                img_path = os.path.join(person_dir, file)
                img = Image.open(img_path).convert("L").resize(image_size)
                images.append(np.asarray(img))
                flat_data.append(np.asarray(img).flatten())
                target.append(label_map[person])
    
    return Bunch(
        data=np.array(flat_data),
        images=np.array(images),
        target=np.array(target),
        target_names=np.array(target_names)
    )

# Cargamos el dataset desde carpeta local
lfw_people = load_custom_lfw(path)

print(f"Imágenes cargadas: {lfw_people.data.shape[0]}")
print(f"Clases (personas): {len(lfw_people.target_names)}")
